In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns


# Linear Data

In [ ]:
# we create 100 observations
n = 100

# this way, our X will go over the range [-3, 3]
x = 6 * np.random.rand(n) - 3
# let's use a quadratic equation
a = 1.5
b= 0.7
noise = np.random.normal(size=n)

y = a * x + b + noise
data = {"x": x, "y": y}
plt.scatter(x, y)

In [ ]:
x.shape, y.shape

## Linear Regression

Note how the model expects a matrix. We need to reshape the data from just 100 numbers to a (100,1) matrix to feed it to the model.

Also note how we don't use a train-test split. You should do so if you are testing a model on actual data.

In [ ]:
%%time
# Linear Regression
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
X = x.reshape(-1, 1)
linreg.fit(X, y)

# make evenly spaced values over the input range
newx = np.linspace(min(X), max(X), 100)
# and predict with those
yhat = linreg.predict(newx)

We can test the performance in different ways. Visual:

In [ ]:
plt.scatter(X, y, label = 'data')
plt.plot(newx, yhat, label = 'model')
plt.legend()

With a $R^2$ score, ranging from 0 to 1:

In [ ]:
linreg.score(X, y)

Or with a rmse, which only makes sense when comparing it to another model. 

In [ ]:
def rmse(yhat, y):
    return np.mean((y-yhat)**2)

rmse(yhat, y)

The SGDregressor give the same result.

In [ ]:
%%time
# SGDRegressor
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(X, y)

newx = np.linspace(min(X), max(X), 100)
# and predict with those
yhat = sgd.predict(newx)

plt.scatter(X, y, label = 'data')
plt.plot(newx, yhat, label = 'model')
plt.legend()

We can try how this works, if we scale the data.
If we do that, we are no longer able to comfortably visualise the data. 
A trick is to simply plot actual vs prediction. It should be close to the diagonal line.

In [ ]:
from sklearn.datasets import make_regression
X5, y5 = make_regression(n_samples=100, n_features=5, noise=25, random_state=42)

sgd.fit(X5, y5)
yhat = sgd.predict(X5)


xmin = np.min([yhat, y5])
xmax = np.max([yhat, y5])
plt.scatter(y5, yhat)
plt.plot([xmin, xmax], [xmin, xmax])

We can also implement mini-batch with this.
It's not really necessary with this small sample, but just to show how it is done:

In [ ]:
# we have 100 observations
X.shape

Let's create 5 groups, and split it up into 5 batches.

In [ ]:
data = pd.DataFrame(x, columns=['x'])
groups = np.random.randint(0, 5, size=n)
data['groups'] = groups
data['y'] = y
batches = [x for _, x in data.groupby('groups')]
batches[0]

After that, we can train partial on each batch.

In [ ]:
sgd = SGDRegressor()

for batch in batches:
    x_ = batch['x'].values.reshape(-1, 1)
    y_ = batch['y'].values
    sgd.partial_fit(x_, y_)

yhat = linreg.predict(newx)

plt.scatter(X, y, label = 'data')
plt.plot(newx, yhat, label = 'model')
plt.legend()

A different apporach is with probabilistic programming. 
The GLM function allows us to specify a formula. "y ~ 1 +  x" means: the basic formula is that we have an $y$ as a target value, and on the other hand a bias and an $x$. Try to find values for the bias and the $x$, such that it predicts the $y$ value.

In [ ]:
import pymc as pm
with pm.Model() as model:
    a = pm.Normal('a')
    b = pm.Normal('b')
    noise = pm.Normal('noise')

    predict = a * x + b + noise
    yhat = pm.Normal("y", mu=predict, observed=y)

    result = pm.sample(2000)

In [ ]:
import arviz as az
az.plot_trace(result)

In [ ]:
estimates = {}
for key in result.posterior.data_vars.keys():
    estimates[key] = result.posterior[key].mean().values
estimates

An advantage of this approach is, that we get confidence intervals.

# Non linear data

So, what if we have non-linear data? let's create some for ourselves.

In [ ]:
# we create 100 observations
n = 100

# this way, our X will go over the range [-3, 3]
x = 6 * np.random.rand(n) - 3
# let's use a quadratic equation
a = 1.5
b = 0.9
c = 2.3
noise = np.random.normal(size=n)

y = c + b * x + a * np.power(x, 2) + noise
data = {"x": x, "y": y}
plt.scatter(x, y)

And try linear regression like before.

In [ ]:
%%time
# Linear Regression
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
X = x.reshape(-1, 1)
linreg.fit(X, y)

# make evenly spaced values over the input range
newx = np.linspace(min(X), max(X), 100)
# and predict with those
yhat = linreg.predict(newx)

plt.scatter(X, y, label = 'data')
plt.plot(newx, yhat, label = 'model')
plt.legend()

In [ ]:
X.shape

well, we could have expected this... The model can only produce lines, so this will never work.

Let's implement the polynomial features.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

poly = PolynomialFeatures(degree = 3, include_bias=True)
X_poly = poly.fit_transform(X)

In [ ]:
X_poly.shape

Note how we expanded the amount of features we have. Because we have only one feature (x) to begin with, this does not explode. But if we would have had more features, using a high degree will very quickly blow up. Test this out for yourself with some artificial data. Change the shape to (100, 15) and first try to predict the shape of the outcome, then see what actually happens to the output.

In [ ]:
test = np.random.rand(100, 5)
p_big = PolynomialFeatures(degree = 3, include_bias=True)
p_big.fit_transform(test).shape

Lets try this

In [ ]:
linreg = LinearRegression()
linreg.fit(X_poly, y)

newx = np.linspace(min(X), max(X), 100)
X_new_poly = poly.fit_transform(newx)
yhat = linreg.predict(X_new_poly)

plt.scatter(X, y, label = 'actual')
plt.plot(newx, yhat, label = 'model')
plt.legend()

That worked! Let's check the coefficients. Do you see how it actually fits the original weights `a` and `b` we used to create this data?

In [ ]:
plt.bar(range(len(linreg.coef_)), linreg.coef_)
a,b

```python
from sklearn import model
mymodel = model(parameters)
mymodel.fit(X, y)
yhat = mymodel.predict(newx)
```

We can use the score function. This give a value between 0 and 1, the $R^2$ value. It is a slight modification of the RMSE and basically tells you how much of the variation of the data can be explained by your model.

What counts as a high enough $R^2$ value depends on the context.

Also note, we score on the train-set. This would be wrong with real data, you should use your test-set for this!

In [ ]:
linreg.score(X_poly, y)

Now, let's add a huge amount of features, just to see what happens.

We will also put everything into a pipeline. This is especially usefull because we want to use the scaler *after* the polynomial features. Do you understand why? If not, discuss this with other students or ask the teacher.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# set the degree to 50 (change it, to see what happens)
degree = 50

polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()

polynomial_regression = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            ("lin_reg", lin_reg),
        ])

polynomial_regression.fit(X, y)

newx = np.linspace(min(X), max(X)+1, 100)
yhat = polynomial_regression.predict(newx)

plt.scatter(X, y, label = 'actual')
plt.plot(newx, yhat, label = 'model')
plt.legend()
plt.axis([-3.2, 3.2, 0, 20])

See what is happening? Well, we might have expected something like this. This is obviously overfitting, our model is much to complex. Now, let's try to add in some regularization again.

First "l1"

In [ ]:
poly_l1 = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            ('sgd', SGDRegressor(penalty='l1', alpha=5))
        ])

poly_l1.fit(X, y)


newx = np.linspace(min(X), max(X), 100)
yhat_l1 = poly_l1.predict(newx)

And then "l2"

In [ ]:
poly_l2 = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            ('sgd', SGDRegressor(penalty='l2', alpha=100))
        ])

poly_l2.fit(X, y)
yhat_l2 = poly_l2.predict(newx)

In [ ]:
plt.scatter(X, y, label = 'actual')
plt.plot(newx, yhat_l1, label = 'l1')
plt.plot(newx, yhat_l2, label = 'l2')
plt.legend()
plt.axis([-3.2, 3.2, 0, 20])

And let's also try elasticnet. To do this, we will need to both tune the l1_ratio and the alpha values.

It is much easier to do this in a parameter grid. Note how we use the double underscores `__` to connect the name of the function in the pipeline to the name of the parameter.

In [ ]:
 [10**i for i in range(-3, 2)]

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ('scaler', StandardScaler()),
     ('sgd', SGDRegressor(penalty='elasticnet'))
])

l1_ratio = [0, 0.01, 0.05, .1, .5, .7, .9, .95, .99, 1]
alphaList = [10**i for i in range(-3, 2)]

param_grid = {'sgd__l1_ratio' : l1_ratio,
                'sgd__alpha' : alphaList}

gridsearch = GridSearchCV(pipe, param_grid=param_grid, cv=5)
gridsearch.fit(X, y)

gridsearch.best_params_

This model picks the l1_ratio. 

To be clear: this is a very unrealistic case. We first set the amount of parameters to 50, which is way too high. And then we try to regulate this again. 

This is like hitting the gas and the brake at the same time, so the model we will produce isn't optimal for prediction. However, this shows you how to do something like this.

Let's try a Support Vector Machine.

In [ ]:
# SVR with kernel
from sklearn.svm import SVR
svr = SVR(C=1, gamma=0.1)
svr.fit(X, y)
yhat = svr.predict(newx)
plt.scatter(X, y, label = 'actual')
plt.plot(newx, yhat, label = 'model')
plt.legend()

The SVM has a natural protection against overfitting, because of the way it tries to find weights that have a big safety margin. The epsilon parameter specifies a tube in which no penalty is assigned with points predicted within a distance epsilon from the actual value.

The gamma value is used to distort the space with the kernel. try to set it much higher (eg 10) to see what happens.

# Probabilistic programming
Next, we can specify a GLM for pymc3. Because our data consists of an `x` and `y` value, we can use those to specify a formula that contains a power.

In [ ]:
x**2

In [ ]:
# baysian regression
with pm.Model() as model:
    a = pm.Normal('a')
    b = pm.Normal('b')
    c = pm.Normal('c')

    predict = a * x**2 + b * x + c
    yhat = pm.Normal("y", mu=predict, observed=y)
    trace = pm.sample(2000)

In [ ]:
az.plot_trace(trace);

You can see we get close to the value. Downsides of this approach are speed and the need to specify a model. For that, you get a confidence interval in return and the flexibility to specify a model, based on your domain knowledge.

In [ ]:
estimates = {}
for key in trace.posterior.data_vars.keys():
    estimates[key] = trace.posterior[key].mean().values
estimates

Last, let's try a random forest regressor. We will tune the depth of the tree.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
param_grid = dict(max_depth = [*range(2, 10, 2)])

gridsearch = GridSearchCV(rfr, param_grid=param_grid, cv=3)
gridsearch.fit(X, y)
gridsearch.best_params_


In [ ]:
yhat = gridsearch.predict(newx)
plt.scatter(X, y, label = 'actual')
plt.plot(newx, yhat, label = 'randomforest')